In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import math
import random
from collections import defaultdict
import keras
from models import TSN
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [2]:
class dataGenerator(keras.utils.Sequence):

    def __init__(self, filepath, batch_size, ffpath, segments = 3, test=False):
        self.filenames = list()
        self.labels = list()
        self.batch_size = batch_size
        self.filepath = filepath
        self.ffpath = ffpath
        self.segments = segments
        
        with open(self.filepath,"r") as f:
            for line in f.readlines():
                arr = line.split(" ")
                self.filenames.append(arr[0])
                self.labels.append(int(arr[1].strip()))
        
    def __len__(self):
        return len(self.filenames)//self.batch_size

    def __getitem__(self, idx):
        batch_x = self.filenames[idx * self.batch_size:min((idx + 1) * self.batch_size, len(self.filenames))]
        batch_y = self.labels[idx * self.batch_size:min((idx + 1) * self.batch_size, len(self.filenames))]

        Xframes = defaultdict(list)
        Y = list()
        Xflows = defaultdict(list)
        for index,each in enumerate(batch_x):
            infopath = os.path.join(self.ffpath,each,"info.txt")
            imgpath = os.path.join(self.ffpath,each,"frames")
            flowspath = os.path.join(self.ffpath,each,"flows")
            f = open(infopath,"r")
            total_frames = int(f.readlines()[0].strip().split(':')[1])
            f.close()
            idxs = []
            base = total_frames//self.segments
            low = 1
            num_frames = 2
            for _ in range(self.segments):
                high = min(low + base, total_frames)
                idxs.extend(np.random.randint(low, high, num_frames))
                low = high + 1 
            frames = self.getFrames(idxs, imgpath)
            flows = self.getFlows(idxs, flowspath)
            
            for i in range(len(frames)):
                Xframes[i%self.segments].append(frames[i])
                        
            for i in range(len(flows)):
                Xflows[i%self.segments].append(flows[i])
                
            Y.extend([batch_y[index]]*(num_frames))
            
        finalX = dict()
        i = 1
        for key in Xframes.keys():
            finalX['input_'+str(i)] = np.array(Xframes[key])
            i += 1
        for key in Xflows.keys():
            finalX['input_'+str(i)] = np.array(Xflows[key])
            i += 1
            
        finalY = {'output':self.one_hot_encode(np.array(Y))}

        return (finalX,finalY)
    
    def one_hot_encode(self,data, classes = 101):
        """
        :param data: data to be one hot encoded
        :return: np array with one hot encoding
        """
        labels = np.zeros((data.size, classes))
        labels[np.arange(data.size), data - 1] = 1
        return labels


    def getFlows(self,idxs, flowspath):

        stack = list()
        for i in idxs:
            f1 = "flow_x_"+str(i)+".jpg"
            f2 = "flow_y_"+str(i)+".jpg"
            grayx = self.readImg(os.path.join(flowspath,f1))
            grayy = self.readImg(os.path.join(flowspath,f2))
            img = np.stack((grayx,grayy),axis = 2)
            img = np.squeeze(img,axis = 3)
            stack.append(img)
            
        return np.array(stack)
    
    def readImg(self,path):
        img = cv2.imread(path)
        grayimg = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        grayimg = np.expand_dims(grayimg,axis = 2)
        return grayimg
    
    
    def getFrames(self,idxs, imgpath):

        stack = list()
        for i in idxs:
            framename = "frame_"+str(i)+".jpg"
            stack.append(self.readImg(os.path.join(imgpath,framename)))
            
        return np.array(stack)

In [3]:
class dataGeneratorOntheFly(keras.utils.Sequence):

    def __init__(self, datapath, filepath, batch_size, segments = 3, test=False):
        self.filenames = list()
        self.labels = list()
        self.batch_size = batch_size
        self.datapath = datapath
        self.filepath = filepath
        self.segments = segments
        
        with open(self.filepath,"r") as f:
            for line in f.readlines():
                arr = line.split(" ")
                self.filenames.append(arr[0])
                self.labels.append(int(arr[1].strip()))
        
    def __len__(self):
        return len(self.filenames)//self.batch_size

    def __getitem__(self, idx):
        batch_x = self.filenames[idx * self.batch_size:min((idx + 1) * self.batch_size, len(self.filenames))]
        batch_y = self.labels[idx * self.batch_size:min((idx + 1) * self.batch_size, len(self.filenames))]

        Xframes = defaultdict(list)
        Y = list()
        Xflows = defaultdict(list)
        for index,each in enumerate(batch_x):
            cap = cv2.VideoCapture(os.path.join(datapath,each))
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            idxs = []
            base = total_frames//self.segments
            low = 1
            for _ in range(self.segments):
                high = min(low + base, total_frames)
                idxs.append(np.random.randint(low, high,1)[0])
                low = high + 1
            frames,flows = self.getFramesandFlows(idxs, cap)
            for i in range(self.segments):
                Xframes[i].append(frames[i])
                
            for i in range(self.segments):
                Xflows[i].append(flows[i])
            
            Y.append(batch_y[index])
            
        finalX = dict()
        i = 1
        for key in Xframes.keys():
            finalX['input_'+str(i)] = np.array(Xframes[key])
            i += 1
        for key in Xflows.keys():
            finalX['input_'+str(i)] = np.array(Xflows[key])
            i += 1
            
        finalY = {'output':self.one_hot_encode(np.array(Y))}

        return (finalX,finalY)
    
    def one_hot_encode(self,data, classes = 101):
        """
        :param data: data to be one hot encoded
        :return: np array with one hot encoding
        """
        labels = np.zeros((data.size, classes))
        labels[np.arange(data.size), data - 1] = 1
        return labels


    def getFramesandFlows(self,idxs,cap):
        
        frames = list()
        flows = list()
        for each in idxs:
            
            cap.set(cv2.CAP_PROP_POS_FRAMES,each)
            _,thisFrame = cap.read()
            cap.set(cv2.CAP_PROP_POS_FRAMES,each-1)
            _,prevFrame = cap.read()
            
            thisFrame = cv2.resize(thisFrame,(224,224))
            prevFrame = cv2.resize(prevFrame,(224,224))
            
            thisFrame  = cv2.cvtColor(thisFrame,cv2.COLOR_BGR2GRAY)
            prevFrame  = cv2.cvtColor(prevFrame,cv2.COLOR_BGR2GRAY)
            
            optical_flow = cv2.optflow.DualTVL1OpticalFlow_create()
            flow = optical_flow.calc(prevFrame, thisFrame, None)
            flow_x = cv2.normalize(flow[...,0],None,0,255,cv2.NORM_MINMAX)
            flow_y = cv2.normalize(flow[...,1],None,0,255,cv2.NORM_MINMAX)
            
            thisFrame = np.expand_dims(thisFrame,axis = 2)
            flow_x = np.expand_dims(flow_x, axis = 2)
            flow_y = np.expand_dims(flow_y, axis = 2)
            
            thisFlow = np.stack((flow_x,flow_y),axis = 2)
            thisFlow = np.squeeze(thisFlow,axis = 3)
            
            frames.append(thisFrame)
            flows.append(thisFlow)
        
        return np.array(frames), np.array(flows)

In [4]:
# folderpath = "../ucfTrainTestlist"
# datapath = "../../UCF-101"
# filename = "trainlist01.txt"
# dg = dataGeneratorNew(datapath,os.path.join(folderpath,filename),batch_size = 128)



folderpath = "../ucfTrainTestlist"
filenameTrain = "custom3Train.txt"
filenameVal = "custom3Val.txt"

filepath = os.path.join(folderpath,filenameTrain)
ffpath = os.path.join("../../VidRecognizer/FramesFlows/custom3")
dgTrain = dataGenerator(filepath,16,ffpath)

filepath = os.path.join(folderpath,filenameVal)
ffpath = os.path.join("../../VidRecognizer/FramesFlows/custom3")
dgVal = dataGenerator(filepath,16,ffpath)

In [5]:
arr = dgTrain.__getitem__(0)

In [6]:
arr[1]['output'].shape

(32, 101)

In [7]:
#Create and Compile model
K.clear_session()
model = TSN()
model.compile(optimizer= keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
#Checkpointing

filepath="../model_checkpoints/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
es = EarlyStopping(monitor='val_acc', mode='max', min_delta=1, patience = 10)
callbacks_list = [checkpoint,es]

In [9]:
# model.fit([np.array(framesSegments[0]),np.array(framesSegments[1]),np.array(framesSegments[2]),np.array(flows[0]),np.array(flows[1]),np.array(flows[2])],yFramesTrueSegments[0],batch_size = 3, epochs = 1)
np.random.seed(0)
#Fit generator
history = model.fit_generator(dgTrain,epochs = 150,validation_data = dgVal, callbacks = callbacks_list)

Instructions for updating:
Use tf.cast instead.
Epoch 1/150
18/18 [==============================] - 105s 6s/step - loss: 7.4750 - acc: 0.0226 - val_loss: 6.6189 - val_acc: 0.0156

Epoch 00001: val_acc improved from -inf to 0.01562, saving model to ../model_checkpoints/weights-improvement-01-0.02.hdf5
Epoch 2/150
18/18 [==============================] - 15s 852ms/step - loss: 6.2627 - acc: 0.0226 - val_loss: 6.9598 - val_acc: 0.0104

Epoch 00002: val_acc did not improve from 0.01562
Epoch 3/150
18/18 [==============================] - 15s 849ms/step - loss: 9.1079 - acc: 0.0208 - val_loss: 10.4063 - val_acc: 0.0104

Epoch 00003: val_acc did not improve from 0.01562
Epoch 4/150
18/18 [==============================] - 15s 848ms/step - loss: 10.6552 - acc: 0.0191 - val_loss: 8.9244 - val_acc: 0.0104

Epoch 00004: val_acc did not improve from 0.01562
Epoch 5/150
18/18 [==============================] - 15s 847ms/step - loss: 8.8282 - acc: 0.0295 - val_loss: 8.7951 - val_acc: 0.0104

Epoch

In [ ]:
print(history.history)